In [1]:
######################################     Modules     #######################################
# MODULES
import sys
from time import process_time
import pickle
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact

sys.path.append('../.')
import verbose_module
sys.path.append('../../../.')
import pIRPgym

path = '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPgym/'
experiments_path = '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/Experiments/First Phase/'

Experiments = [1,2,3,4,5]
# Replicas = [i for i in range(1,6)]
Replicas = {i:[j for j in range(1,6)] for i in range(1,5)}
Replicas.update({5:[1,2,3]})


alphas = [0.1,0.2,0.4,0.6,0.8]
time_limits = [1,30,60,300,1800,3600]

init_times = {1:0.1,30:1,60:3,300:5,1800:5,3600:10}

Policies = ['NN','RCL']+[f'CG_{time_limit}' for time_limit in time_limits]+\
           [f'CG_{time_limit}_{alpha}' for time_limit in time_limits for alpha in alphas]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11


In [3]:
Experiments_results = {('Instance','M/K'):[5,10,15,20,40]}
Experiments_results.update({(policy,item):[] for policy in Policies for item in ['eff','gaps']})

for experiment in Experiments:

    reliability = {policy:[] for policy in Policies}
    gaps = {policy:[] for policy in Policies}

    for replica in Replicas[experiment]:
        with open(experiments_path+f'Experiment {experiment}/Replica {replica}/instance_information.pkl', 'rb') as file:
            inst_info = pickle.load(file)

        results = dict()
        for policy in Policies:
            with open(experiments_path+f'Experiment {experiment}/Replica {replica}/{policy}.pkl','rb') as file:
                results[policy] = pickle.load(file)

        for t in range(inst_info['inst_gen'].T):
            best_performance = min([results['NN'][t][1]]+\
                                   [results['RCL'][t][0]]+\
                                   [results[policy][t][1] for policy in Policies[2:]])

            for policy in Policies:
                loc = 1 if policy!='RCL' else 0
                
                if results[policy][t][loc] - best_performance < 0.1:
                    reliability[policy].append(1)
                else:
                    reliability[policy].append(0)
                    gaps[policy].append((results[policy][t][loc]-best_performance)/best_performance)

    # Storing the results
    for policy in Policies:
        Experiments_results[(policy,'eff')].append(round(sum(reliability[policy])/len(reliability[policy]),4))
        if len(gaps[policy]) > 0:
            Experiments_results[(policy,'gaps')].append(round(sum(gaps[policy])/len(gaps[policy]),4))
        else:
            Experiments_results[(policy,'gaps')].append(0)

df = pd.DataFrame(Experiments_results)
df.index = Experiments
df = df.style.format(dict.fromkeys(list(df.columns)[1:],"{:.2%}"))
display(df)

In [6]:
sizes = {1:5,2:10,3:15,4:20,5:40}
E_x_values = [f'Exp{exp} (M/K={size})' for exp,size in sizes.items()]


def plot_thing(indicator,runtime):
    E_results = dict()
    policies = ['NN','RCL',f'CG_{runtime}']+[f'CG_{runtime}_{alpha}' for alpha in alphas]
    for policy in policies:
        E_results[policy] ={'eff':Experiments_results[str(policy),'eff'],'gaps':Experiments_results[str(policy),'gaps']} 
    
    pIRPgym.Visualizations.RoutingV.plot_indicator_evolution(E_results,indicator,x_axis='Experiments',x_values=E_x_values)

interact(plot_thing,indicator=widgets.Dropdown(options=['eff','gaps'],value='eff',description='Indicator:'),
         runtime=widgets.Dropdown(options=[1,30,60,300,1800,3600],value=1,description='Run time:'))

interactive(children=(Dropdown(description='Indicator:', options=('eff', 'gaps'), value='eff'), Dropdown(descr…

<function __main__.plot_thing(indicator, runtime)>

# By suppliers


In [7]:
Suppliers_results = {('Instance','M/K'):[],('Instance','Number'):[]}
Suppliers_results.update({(policy,item):[] for policy in Policies for item in ['eff','gaps']})

reliability = {policy:dict() for policy in Policies}
gaps = {policy:dict() for policy in Policies}

for experiment in Experiments:
    for replica in Replicas[experiment]:
        # Upload results
        with open(experiments_path+f'Experiment {experiment}/Replica {replica}/instance_information.pkl', 'rb') as file:
            inst_info = pickle.load(file)

        results = dict()
        for policy in Policies: 
            with open(experiments_path+f'Experiment {experiment}/Replica {replica}/{policy}.pkl', 'rb') as file:
                results[policy] = pickle.load(file)

        
        
        # For each day determine the best score
        for t in range(inst_info['inst_gen'].T):
            num_suppliers = len([node for route in results['NN'][t][0] for node in route if node != 0])
            if num_suppliers not in reliability['NN'].keys():
                for policy in Policies:
                    reliability[policy][num_suppliers] = list()
                    gaps[policy][num_suppliers] = list()

            best_performance = min([results[policy][t][1] for policy in Policies if policy != 'RCL']+\
                                   [results['RCL'][t][0]])

            # if results[t][1] - best_performance < 0.01:
            #     reliability['CG'][num_suppliers].append(1)
            # else:
            #     reliability['CG'][num_suppliers].append(0)
            #     gaps['CG'][num_suppliers].append((CG_results[t][1]-best_performance)/best_performance)

            for policy in Policies: 
                pos = 1 if policy != 'RCL' else 0
                if results[policy][t][pos] - best_performance < 0.1:
                    reliability[policy][num_suppliers].append(1)
                else:
                    reliability[policy][num_suppliers].append(0)
                    gaps[policy][num_suppliers].append((results[policy][t][pos]-best_performance)/best_performance)

                    

# Storing the results
supplier_categories = sorted(list(set(reliability['NN'].keys())))
Suppliers_results[('Instance','M/K')] = supplier_categories

for sup_num in supplier_categories:
    Suppliers_results[('Instance','Number')].append(len(reliability["NN"][sup_num]))


    for policy in Policies:
        Suppliers_results[(str(policy),'eff')].append(round(sum(reliability[policy][sup_num])/len(reliability[policy][sup_num]),4))
        if len(gaps[policy][sup_num]) > 0:
            Suppliers_results[(str(policy),'gaps')].append(round(sum(gaps[policy][sup_num])/len(gaps[policy][sup_num]),4))
        else:
            Suppliers_results[(str(policy),'gaps')].append(0)

df = pd.DataFrame(Suppliers_results)
df.index = supplier_categories
# df.columns = ['M/K','Regular', 'al 0.1','al 0.2','al 0.4','al 0.6']
df = df.style.format(dict.fromkeys(list(df.columns)[2:],"{:.2%}"))
display(df)

In [9]:
def plot_thing_2(indicator,display_range,runtime):
    policies = ['NN','RCL',f'CG_{runtime}']+[f'CG_{runtime}_{alpha}' for alpha in alphas]

    results = dict()
    for policy in Policies:
        results[policy] = {'eff':Suppliers_results[policy,'eff']}
        results[policy].update({'gaps':Suppliers_results[policy,'gaps']})

    display_num_suppliers = list()
    display_results = dict()
    for policy in policies:
        display_results[policy] = {'eff':list()}
        display_results[policy].update({'gaps':list()})

    for i,num in enumerate(supplier_categories):
        if num >= display_range[0] and num <= display_range[1]:
            display_num_suppliers.append(num)
            for policy in policies:
                display_results[policy]['eff'].append(results[policy]['eff'][i])
                display_results[policy]['gaps'].append(results[policy]['gaps'][i])

    pIRPgym.Visualizations.RoutingV.plot_indicator_evolution(display_results,indicator,x_axis='Suppliers',
                                                             x_values=display_num_suppliers)

interact(plot_thing_2,
         indicator=widgets.Dropdown(options=['eff','gaps'],value='eff',
                                    description='Indicator:'),
         display_range=widgets.IntRangeSlider(  value=[1,40],min=1,max=40,step=1,
                                                description='Suppliers:',continuous_update=False,
                                                orientation='horizontal',readout=True,
                                                readout_format='d'),
         runtime=widgets.Dropdown(options=[1,30,60,300,1800,3600],value=1,description='Run time:'))

interactive(children=(Dropdown(description='Indicator:', options=('eff', 'gaps'), value='eff'), IntRangeSlider…

<function __main__.plot_thing_2(indicator, display_range, runtime)>

In [6]:
with open(experiments_path+f'Experiment 4/Replica 1/CG_3600.pkl','rb') as file:
        results = pickle.load(file)

In [7]:
print(f'FO: {results[7][1]}')
print(f'Init cols: {results[7][4][0]}')
print(f'CG cols: {results[7][4][1]}')

FO: 2936.92635757522
Init cols: 0
CG cols: 100


In [8]:
with open(experiments_path+f'Experiment 4/Replica 1/CG_3600_0.8.pkl','rb') as file:
        results2 = pickle.load(file)

In [9]:
print(f'FO: {results2[7][1]}')
print(f'Init cols: {results2[7][4][0]}')
print(f'CG cols: {results2[7][4][1]}')

FO: 10268.802214227899
Init cols: 12999
CG cols: 8
